In [3]:
#install pymupdf or other necessary libraries

!pip install PyMuPDF

In [4]:
#Import libraries
import os
import io
from PIL import Image
import fitz
import shutil

# Single pdf image conversion to png

In [34]:
def convert_pdf_to_images(pdf_path, output_folder, output_format="png"):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Open the PDF file
    pdf_document = fitz.open(pdf_path)
    
    # Initialize image counter
    image_count = 0
    
    # Loop through each page
    for page_number in range(pdf_document.page_count):
        # Get the page
        page = pdf_document.load_page(page_number)
        
        # Get the images from the page
        img_list = page.get_images(full=True)
        
        # Iterate through each image on the page
        for img_index, img in enumerate(img_list):
            # Get the byte of the image
            img_byte = pdf_document.extract_image(img[0])["image"]
            
            # Convert the bytes to PIL image
            pil_img = Image.open(io.BytesIO(img_byte))
            
            # Save image 
            output_file = f"{os.path.splitext(os.path.basename(pdf_path))[0]}_page_{page_number + 1}_img_{img_index + 1}.{output_format}"
            output_path = os.path.join(output_folder, output_file)
            
            # Check if the image file already exists in the output folder
            if os.path.exists(output_path):
                print(f"Image {output_file} already exists. Skipping...")
                continue
            
            # Save the image if it doesn't exist
            pil_img.save(output_path)
            
            # Increment image counter
            image_count += 1
    
    # Close the PDF document
    pdf_document.close()
    
    print(f"Total {image_count} images were extracted from {pdf_path}")

In [ ]:
# Example usage
pdf_path = r'C:\path\to\your\file.pdf'
output_folder = r'C:\path\to\output\folder'
convert_pdf_to_images(pdf_path, output_folder, output_format="png")

# Single folder Conversion

In [35]:
def convert_pdfs_to_images(input_folder, output_folder, output_format="png"):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for pdf_file in os.listdir(input_folder):
        if not pdf_file.endswith(".pdf"):
            continue

        pdf_path = os.path.join(input_folder, pdf_file)
        document = fitz.open(pdf_path)

        for page_num in range(len(document)):
            page = document.load_page(page_num)
            pix = page.get_pixmap()

            pil_img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            
            # Construct the output path
            output_path = os.path.join(output_folder, f"{os.path.splitext(pdf_file)[0]}_page_{page_num + 1}_img_1.{output_format}")
            
            # Create the directory if it doesn't exist
            output_dir = os.path.dirname(output_path)
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            
            # Save the image if it doesn't exist
            pil_img.save(output_path)

In [ ]:
# Example usage
input_folder = "where_the_pdf_image_lies"
output_folder = "desired_output_folder"
convert_pdfs_to_images(input_folder, output_folder, output_format="png")

# Mass conversion from subfolders containing the pdf images

In [ ]:
#Creating subfolders of same name with the input folder containing the pdf images somwhere else to save the png images
def create_subfolders(source_folder, destination_folder):
    # Get the list of subfolder names from the source folder
    subfolders = [name for name in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, name))]
    
    # Iterate through each subfolder name
    for subfolder in subfolders:
        # Create the corresponding subfolder in the destination folder
        subfolder_path = os.path.join(destination_folder, subfolder)
        os.makedirs(subfolder_path, exist_ok=True)  # Create subfolder if it doesn't exist already

# Example usage:
source_folder_external = "/path_to_folder_containing_pdf_images"  # Specify the full path to the source folder 
destination_folder = "destiantion_to_store_the_png_images"    # Specify the path to the destination folder

create_subfolders(source_folder_external, destination_folder)

In [ ]:
# #Function to set a limit on the folder size incase it contains other uncesssary folders you do not need to convert
# def is_convertible(pdf_path, max_size=3000):
#     # Get the size of the PDF file in bytes
#     file_size = os.path.getsize(pdf_path)
    
#     if file_size > max_size * 1024: 
#         # Convert max_size from KB to bytes
#         print(f"Ignoring {pdf_path} because it is larger than {max_size} KB")
#         return False 
#     return True

In [ ]:
#mASS eXTRACTION, that is looping through a subfolders in a larger folder containing the pdf images
import os
import io
from PIL import Image
import fitz  # PyMuPDF

def convert_pdfs_to_images(input_folder, output_folder, output_format="png"):
    for root, _, files in os.walk(input_folder):
        # Get the relative path to maintain folder structure in the output
        relative_path = os.path.relpath(root, input_folder)
        output_subfolder = os.path.join(output_folder, relative_path)
        
        # Create the output subfolder if it doesn't exist
        if not os.path.exists(output_subfolder):
            os.makedirs(output_subfolder)

        for pdf_file in files:
            if pdf_file.endswith(".pdf"):
                pdf_path = os.path.join(root, pdf_file)
                pdf_document = fitz.open(pdf_path)
                image_count = 0

                for page_number in range(pdf_document.page_count):
                    page = pdf_document.load_page(page_number)
                    img_list = page.get_images(full=True)

                    for img_index, img in enumerate(img_list):
                        img_byte = pdf_document.extract_image(img[0])["image"]
                        pil_img = Image.open(io.BytesIO(img_byte))
                        output_file = f"{os.path.splitext(pdf_file)[0]}_page_{page_number + 1}_img_{img_index + 1}.{output_format}"
                        output_path = os.path.join(output_subfolder, output_file)
                        
                        pil_img.save(output_path)
                        image_count += 1

                pdf_document.close()
                print(f"Total {image_count} images were extracted from {pdf_file}")

In [ ]:
# Example usage
input_folder = "path_to_the_folders_containing_subfolders"
output_folder = "desired_output_folder"
convert_pdfs_to_images(input_folder, output_folder, output_format="png")